In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import warnings
import datetime
import argparse
import threading
import multiprocessing
import os

from tqdm import tqdm

from GBP.data import DataGenerator
from GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP
from GBP.utilities import HiddenPrints
from GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

In [ ]:
dataset = 'input_MITb_g2o'
filepath_n = 'GBP/Raw_Datasets/gbp_data'
factor_data = os.path.join(filepath_n, f'{dataset}_factor_data.txt')
marginal_data = os.path.join(filepath_n, f'{dataset}_marginal_data.txt')

A, b = data_gen.fetch_SLAM_dataset(file_path_factor=factor_data, file_path_marginal=marginal_data)

In [ ]:
sync_convergence_threshold = 1*10**-8

NODE_UPDT_PE = 100
PEs = 1

ASYNC_ITER = 10

#### Sync

In [ ]:
sync_convergence_threshold = 1*10**-8 # convergence threshold
convergence_type = 'all' #all
simulation_convergence_threshold = 1*10**-8

P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, 
                                                                                                      b, 
                                                                                                      max_iter=100_000, 
                                                                                                      mae=True if convergence_type == 'mae' else False, 
                                                                                                      convergence_threshold=simulation_convergence_threshold,
                                                                                                      show=True)
final_mean = list(mu_i)

### Residual - Probability Method (UPDT) - previous T max

In [ ]:
async_convergence_threshold = 1 * 10**-5

sum_of_iterations = 0
num_iterations = ASYNC_ITER

# Track the valid iterations for averaging
valid_iterations = 0

for it in range(num_iterations):
    while True:
        # Run the hardware-accelerated residual function
        P_i, mu_i, iteration, prev_t_max, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX(A, b, SCALING=False, SHUFFLE_STREAM=False, CURR_T_MAX=False, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=True, convergence_threshold=async_convergence_threshold, show=False)
        
        # Check if iteration is valid
        if iteration <= 10000:
            sum_of_iterations += iteration
            valid_iterations += 1
            break  # Exit the while loop and proceed to the next for loop iteration
        else:
            print(f"Iteration {it+1} exceeded 10000 steps. Repeating...")

    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# Calculate the average of valid iterations
if valid_iterations > 0:
    average_iterations = sum_of_iterations / valid_iterations
else:
    average_iterations = 0

print(f"AVE. ASYNC ITERATIONS = {average_iterations}")

print(PADDING_ANALYTICS)
print(sum(PADDING_ANALYTICS))
print(sum(PADDING_ANALYTICS)/len(PADDING_ANALYTICS))

In [ ]:
async_convergence_threshold = 1 * 10**-5

sum_of_iterations = 0
num_iterations = ASYNC_ITER

# Track the valid iterations for averaging
valid_iterations = 0

for it in range(num_iterations):
    while True:
        # Run the hardware-accelerated residual function
        P_i, mu_i, iteration, prev_t_max, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP(A, b, SCALING=False, SHUFFLE_STREAM=False, CURR_T_MAX=False, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
        
        # Check if iteration is valid
        if iteration <= 1000:
            sum_of_iterations += iteration
            valid_iterations += 1
            break  # Exit the while loop and proceed to the next for loop iteration
        else:
            print(f"Iteration {it+1} exceeded 1000 steps. Repeating...")

    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# Calculate the average of valid iterations
if valid_iterations > 0:
    average_iterations = sum_of_iterations / valid_iterations
else:
    average_iterations = 0

print(f"AVE. ASYNC ITERATIONS = {average_iterations}")

print(PADDING_ANALYTICS)
print(sum(PADDING_ANALYTICS))
print(sum(PADDING_ANALYTICS)/len(PADDING_ANALYTICS))